In [1]:
# !pip install tensorflow opencv-python matplotlib

In [7]:
# !pip install mediapipe

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()


In [11]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis = 0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    # print(keypoints_with_scores)
    img_resized = np.array(input_image).astype(np.uint8)[0]
    keypoints_for_resized = keypoints_with_scores.copy()
    keypoints_for_resized[:, 0] *= img_resized.shape[1]
    keypoints_for_resized[:, 1] *= img_resized.shape[0]
    draw_keypoints(img_resized, keypoints_for_resized, 0.4)
    cv2.imwrite("image_with_keypoints_resized.png", img_resized)

    orig_w, orig_h = frame.shape[:2]
    M = get_affine_transform_to_fixed_sizes_with_padding((orig_w, orig_h), (192, 192))
    # M has shape 2x3 but we need square matrix when finding an inverse
    M = np.vstack((M, [0, 0, 1]))
    M_inv = np.linalg.inv(M)[:2]
    xy_keypoints = keypoints_with_scores[:, :2] * 192
    xy_keypoints = cv2.transform(np.array([xy_keypoints]), M_inv)[0]
    keypoints_with_scores = np.hstack((xy_keypoints, keypoints_with_scores[:, 2:]))

    # Rendering
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    cv2.imwrite("image_with_keypoints_original.png", frame)
    # rendering
    """draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)"""
    
    cv2.imshow("Movenet Lightning", frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

IndexError: index 1 is out of bounds for axis 1 with size 1

In [10]:
def get_affine_transform_to_fixed_sizes_with_padding(size, new_sizes):
    width, height = new_sizes
    scale = min(height / float(size[1]), width / float(size[0]))
    M = np.float32([[scale, 0, 0], [0, scale, 0]])
    M[0][2] = (width - scale * size[0]) / 2
    M[1][2] = (height - scale * size[1]) / 2
    return M

In [1]:
keypoints_with_scores[0][0][4][:2]

NameError: name 'keypoints_with_scores' is not defined

In [13]:
# nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, 
# left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle

right_eye = keypoints_with_scores[0][0][2]
right_eye

# this one is right eye

array([0.47637603, 0.51566744, 0.6938343 ], dtype=float32)

In [14]:
# 480, 640 is frame shape
np.array(right_eye[:2]*[480,640]).astype(int)

array([228, 330])

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) # confidence metric c, dont transform that
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) #.circle creates circle, 4 is size, 255 is color green, -1 is fill circle

In [14]:
# pip install --upgrade tensorflow
# pip install --upgrade numpy
# pip install --upgrade scipy
# pip install --upgrade h5py

In [4]:
# nose, left eye, right eye, left ear, right ear, 
# left shoulder, right shoulder, left elbow, right elbow, 
# left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle

EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)